# Part 1: Transformers

##Task 1 (50 points):
In this task you should work with the Facebook BART model
(https://huggingface.co/docs/transformers/en/model_doc/bart) to provide text summarization
of news articles. Text summarization in Natural Language Processing (NLP) is a technique that
breaks down long texts into sentences or paragraphs, while retaining the text's meaning and
extracting important information. Pick any one dataset of your choice from
https://data.world/datasets/news-dataset.

I am using this news dataset: https://data.world/crawlfeeds/cnbc-news-dataset
It is about CNBC News.

In [1]:
import pandas as pd

df = pd.read_csv('cnbc_news_datase.csv')
df.dropna(subset=['description', 'title'], inplace=True)
df.head()

,title,url,published_at,author,publisher,short_description,keywords,header_image,raw_description,description,scraped_at
0,Santoli’s Wednesday market notes: Could Septem...,https://www.cnbc.com/2021/09/29/santolis-wedne...,2021-09-29T17:09:39+0000,Michael Santoli,CNBC,"This is the daily notebook of Mike Santoli, CN...","cnbc, Premium, Articles, Investment strategy, ...",https://image.cnbcfm.com/api/v1/image/10694960...,"<div class=""group""><p><em>This is the daily no...","This is the daily notebook of Mike Santoli, CN...",2021-10-30 14:11:23.709372
3,US Moves Closer to Becoming A Major Shareholde...,https://www.cnbc.com/2009/04/22/us-moves-close...,2009-04-22T19:49:03+0000,Michelle Caruso-Cabrera,CNBC,The US government is increasingly likely to co...,"cnbc, Articles, General Motors Co, Business Ne...",https://image.cnbcfm.com/api/v1/image/24947979...,"<div class=""group""><p>The US government is inc...",The US government is increasingly likely to co...,2021-10-30 14:11:24.261143
4,Trump: 'Mission accomplished' on 'perfectly ex...,https://www.cnbc.com/2018/04/14/trump-mission-...,2018-04-14T14:59:04+0000,Javier E. David,CNBC,NaN,"cnbc, Articles, George W. Bush, Vladimir Putin...",https://image.cnbcfm.com/api/v1/image/10513177...,"<div class=""group""></div>,<div class=""group""><...",President Donald Trump hailed the U.S.-led int...,2021-10-30 14:11:24.489490
5,Chevron CEO Watson says he supports Trump on t...,https://www.cnbc.com/2017/03/07/chevron-ceo-wa...,2017-03-07T23:07:14+0000,Lauren Thomas,CNBC,Chevron Chief Executive John Watson told CNBC ...,"cnbc, Articles, White House, Oil and Gas, Chev...",https://image.cnbcfm.com/api/v1/image/10322670...,"<div class=""group""><p><a href=""//www.cnbc.com/...",Chevron Chief Executive John Watson told CNBC ...,2021-10-30 14:11:24.736488
7,European stocks close higher on supportive Fed...,https://www.cnbc.com/2020/12/17/european-stock...,2020-12-17T06:08:38+0000,"Elliot Smith,Holly Ellyatt",CNBC,LONDON — European stocks closed higher on Thur...,"cnbc, Articles, World economy, World Markets, ...",https://image.cnbcfm.com/api/v1/image/10681242...,"<div class=""group""><p>LONDON — European stocks...",LONDON — European stocks closed higher on Thur...,2021-10-30 14:11:24.834045


In [2]:
from sklearn.model_selection import train_test_split
# Split the DataFrame into train and test sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Display the first few rows of the train DataFrame to verify
print(train_df.head())

# If you need to check the size of each set to confirm the split
print(f"Train set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")

                                                 title  \
561                   The Blogger response to Bug Labs   
244  US court upholds Obama-era retirement advice rule   
530       Will the US Debt Ceiling be Raised by Aug 2?   
78                    5 Stocks Insiders Love Right Now   
193  U.S. lawmakers agree Big Tech has too much pow...   

                                                   url  \
561  https://www.cnbc.com/2008/01/11/the-blogger-re...   
244  https://www.cnbc.com/2017/02/09/us-court-uphol...   
530  https://www.cnbc.com/2011/07/01/will-the-us-de...   
78   https://www.cnbc.com/2012/07/18/5-stocks-insid...   
193  https://www.cnbc.com/2020/07/30/us-lawmakers-a...   

                 published_at        author publisher  \
561  2008-01-11T20:32:18+0000           NaN      CNBC   
244  2017-02-09T10:03:23+0000           NaN      CNBC   
530  2011-07-01T08:15:16+0000           NaN      CNBC   
78   2012-07-18T16:51:47+0000           NaN      CNBC   
193  2020-07-30T1

In [3]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [6]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# You will need to preprocess your data to convert the text and summaries into a format suitable for the model
# This involves tokenizing the texts and summaries, and then creating a PyTorch dataset from them

# Example of tokenizing a text and its summary
inputs = tokenizer(train_df['description'].tolist(), max_length=512, truncation=True, padding="max_length", return_tensors="pt")
with tokenizer.as_target_tokenizer():
    labels = tokenizer(train_df['title'].tolist(), max_length=128, truncation=True, padding="max_length", return_tensors="pt").input_ids

# You would then create a custom dataset from these tokenized inputs and labels


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd

# Example: Load a CSV file into a DataFrame
# df = pd.read_csv('path_to_your_dataset.csv')

# For demonstration, let's split a DataFrame into training and testing sets
# train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

class SummaryDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_length = max_length
        self.texts = self.data.description
        self.summaries = self.data.title

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        summary = str(self.summaries.iloc[idx])

        # Tokenize the text and summary
        inputs = self.tokenizer(text, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")
        targets = self.tokenizer(summary, padding="max_length", truncation=True, max_length=128, return_tensors="pt").input_ids

        inputs_ids = inputs.input_ids.squeeze()
        attention_mask = inputs.attention_mask.squeeze()
        targets_ids = targets.squeeze()

        return {
            "input_ids": inputs_ids,
            "attention_mask": attention_mask,
            "labels": targets_ids
        }

# Initialize the BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Create the datasets
train_dataset = SummaryDataset(train_df, tokenizer)
test_dataset = SummaryDataset(test_df, tokenizer)

# Example of using DataLoader to iterate through the dataset
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Example iteration over the DataLoader (in your training loop)
# for batch in train_dataloader:
#     input_ids = batch['input_ids']
#     attention_mask = batch['attention_mask']
#     labels = batch['labels']
#     # Pass data through your model here


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

In [11]:
!pip install transformers[torch]

In [7]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

## Let's try with batch size 4 first

In [15]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments

In [7]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=4,   # batch size per device during training
    per_device_eval_batch_size=4,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Assuming you've created a PyTorch dataset for training
    eval_dataset=test_dataset     # Assuming you've created a PyTorch dataset for evaluation
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=402, training_loss=1.5150699141013682, metrics={'train_runtime': 157.2093, 'train_samples_per_second': 10.171, 'train_steps_per_second': 2.557, 'total_flos': 1732600129388544.0, 'train_loss': 1.5150699141013682, 'epoch': 3.0})

In [8]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00


In [14]:
def generate_summary(model, tokenizer, text, max_input_length=512, max_output_length=150):
    model.eval()
    device = model.device  # Use the model's device
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=max_input_length, truncation=True).to(device)
    summary_ids = model.generate(inputs, max_length=max_output_length, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


# Generate predicted summaries for each article in the test dataset
predicted_summaries = [generate_summary(trainer.model, tokenizer, text) for text in test_df['description'].tolist()]

# Prepare reference summaries
# For ROUGE, you typically need each reference summary as a single string in a list
reference_summaries_rouge = list(test_df['title'])

# For BLEU, you need the reference summaries tokenized in a list of lists
# This is a simple tokenization process for illustration; you might need a more sophisticated approach depending on your requirements
reference_summaries_bleu = [[summary.split()] for summary in test_df['title']]


In [15]:
print(reference_summaries_bleu)

[[['Chinese', 'Retail', 'Sales', 'Surge,', 'Partly', 'on', 'Inflation']], [['Baseball', 'Hall', 'of', 'Fame', 'Presented', 'With', 'One-of-a-Kind', 'Babe', 'Ruth', 'Painting', 'Donated', 'by', "Aaron's,", 'Inc.']], [['Buffalo', 'Wild', 'Wings', 'execs', 'have', "'not", 'put', 'their', 'money', 'where', 'their', 'mouth', "is,'", 'activist', 'says']], [["Goldman's", 'Best', 'Energy', 'Stocks', 'for', 'the', 'Next', 'Six', 'Months']], [['Nadja', 'Swarovski:', 'The', 'luxury', 'sector', 'has', 'been', '‘incredibly', 'resilient’', 'amid', 'the', 'pandemic']], [['S&P', '500', 'closes', 'at', 'new', 'record', 'as', 'chipmakers', 'get', 'a', 'boost', 'from', 'US-China', 'trade', 'truce']], [['Central', 'banks', 'have', 'almost', 'eliminated', 'recessions,', 'venture', 'capitalist', 'Palihapitiya', 'says']], [['Trump', 'sued', 'by', 'Democratic', 'lawmakers', 'over', 'foreign', 'state', 'payments', 'to', 'his', 'businesses']], [['Money', 'In', 'Motion:', 'Euro', 'Vs.', 'Dollar']], [['Britain', 

In [11]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=060b0a1e69e514f92fd6a582a7692c77258e003d81c3324974cc7b1540c05c34
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [16]:
from datasets import load_metric

rouge = load_metric('rouge')
bleu = load_metric('bleu')

# Generate summaries for the test set
# Calculate BLEU and ROUGE scores using the datasets library
# Note: Actual implementation requires generating summaries with the model and comparing them to the ground truth

results_rouge = rouge.compute(predictions=predicted_summaries, references=reference_summaries_rouge)
results_bleu = bleu.compute(predictions=[pred.split() for pred in predicted_summaries], references=reference_summaries_bleu)

print("ROUGE Score:", results_rouge)
print("BLEU Score:", results_bleu)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major r

ROUGE Score: {'rouge1': AggregateScore(low=Score(precision=0.14375999662668354, recall=0.422550009371362, fmeasure=0.21183329125956313), mid=Score(precision=0.1678065875484685, recall=0.48557159351083495, fmeasure=0.2457842885886731), high=Score(precision=0.1923199986839515, recall=0.5501031171772273, fmeasure=0.2795415383596427)), 'rouge2': AggregateScore(low=Score(precision=0.04925322199774831, recall=0.1510881081333384, fmeasure=0.07239795521852747), mid=Score(precision=0.06427091813403499, recall=0.1956946367830656, fmeasure=0.09449137199342601), high=Score(precision=0.08141730306988162, recall=0.2423286519265621, fmeasure=0.11854975062907387)), 'rougeL': AggregateScore(low=Score(precision=0.11376716305446143, recall=0.3386193235764635, fmeasure=0.16779622260938748), mid=Score(precision=0.13339944932164996, recall=0.3902782768630292, fmeasure=0.19591913707493985), high=Score(precision=0.153249315525203, recall=0.44285767984452185, fmeasure=0.2235585811632259)), 'rougeLsum': Aggrega

In [17]:
# Function to print ROUGE scores in a formatted way
def print_rouge_scores(rouge_results):
    print("ROUGE Scores:")
    for key, value in rouge_results.items():
        print(f"{key}:")
        print(f"  Precision (mid): {value.mid.precision:.4f}")
        print(f"  Recall (mid): {value.mid.recall:.4f}")
        print(f"  F1-score (mid): {value.mid.fmeasure:.4f}\n")

# Function to print BLEU score and details in a formatted way
def print_bleu_score(bleu_result):
    print("BLEU Score:")
    print(f"  BLEU: {bleu_result['bleu']:.4f}")
    print("  Precisions:")
    for i, p in enumerate(bleu_result['precisions'], start=1):
        print(f"    N-{i}: {p * 100:.2f}%")
    print(f"  Brevity Penalty: {bleu_result['brevity_penalty']:.4f}")
    print(f"  Length Ratio: {bleu_result['length_ratio']:.4f}")
    print(f"  Translation Length: {bleu_result['translation_length']}")
    print(f"  Reference Length: {bleu_result['reference_length']}")

print_rouge_scores(results_rouge)
print_bleu_score(results_bleu)


ROUGE Scores:
rouge1:
  Precision (mid): 0.1678
  Recall (mid): 0.4856
  F1-score (mid): 0.2458

rouge2:
  Precision (mid): 0.0643
  Recall (mid): 0.1957
  F1-score (mid): 0.0945

rougeL:
  Precision (mid): 0.1334
  Recall (mid): 0.3903
  F1-score (mid): 0.1959

rougeLsum:
  Precision (mid): 0.1331
  Recall (mid): 0.3910
  F1-score (mid): 0.1955

BLEU Score:
  BLEU: 0.0294
  Precisions:
    N-1: 13.07%
    N-2: 4.51%
    N-3: 1.86%
    N-4: 0.68%
  Brevity Penalty: 1.0000
  Length Ratio: 2.8794
  Translation Length: 1791
  Reference Length: 622


## Here, we try batch size = 2

In [18]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=2,   # batch size per device during training
    per_device_eval_batch_size=2,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Assuming you've created a PyTorch dataset for training
    eval_dataset=test_dataset     # Assuming you've created a PyTorch dataset for evaluation
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,1.324700


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=801, training_loss=0.8703548274236672, metrics={'train_runtime': 253.7474, 'train_samples_per_second': 6.302, 'train_steps_per_second': 3.157, 'total_flos': 1732600129388544.0, 'train_loss': 0.8703548274236672, 'epoch': 3.0})

In [19]:
predicted_summaries = [generate_summary(trainer.model, tokenizer, text) for text in test_df['description'].tolist()]

print("Scores under Batch Size 2")
results_rouge = rouge.compute(predictions=predicted_summaries, references=reference_summaries_rouge)
results_bleu = bleu.compute(predictions=[pred.split() for pred in predicted_summaries], references=reference_summaries_bleu)

print_rouge_scores(results_rouge)
print_bleu_score(results_bleu)

Scores under Batch Size 2
ROUGE Scores:
rouge1:
  Precision (mid): 0.1680
  Recall (mid): 0.4408
  F1-score (mid): 0.2375

rouge2:
  Precision (mid): 0.0646
  Recall (mid): 0.1834
  F1-score (mid): 0.0932

rougeL:
  Precision (mid): 0.1394
  Recall (mid): 0.3764
  F1-score (mid): 0.1987

rougeLsum:
  Precision (mid): 0.1389
  Recall (mid): 0.3751
  F1-score (mid): 0.1985

BLEU Score:
  BLEU: 0.0324
  Precisions:
    N-1: 13.28%
    N-2: 4.79%
    N-3: 2.17%
    N-4: 0.80%
  Brevity Penalty: 1.0000
  Length Ratio: 2.7122
  Translation Length: 1687
  Reference Length: 622


We can see that for both batch sizes, BART does not perform good summary quatlity. It's probably because of small finetuning epochs and limited GPU sources while running on colab. when Batch size = 4, the funetined BART does not have pretty good summary quality.

#Part 2: Reinforcement Learning
## Task 2(25 points):
We discussed how we can formulate RL problems as an MDP. Describe any
real-world application that can be formulated as an MDP. Describe the state space, action
space, transition model, and rewards for that problem. You do not need to be precise in the
description of the transition model and reward (no formula is needed). Qualitative description
is enough.

## Answer:

A real-world application that can be formulated as a Markov Decision Process (MDP) is autonomous driving. In this scenario, the goal is for an autonomous vehicle to navigate through traffic to a destination safely and efficiently. Let's break down the components of this MDP:

State Space
The state space represents all possible situations that the autonomous vehicle could encounter. This includes a wide range of variables, such as:

1. The vehicle's current speed and direction.
2. The position of the vehicle relative to important objects, such as other vehicles, pedestrians, traffic lights, and road signs.
3. The vehicle's current location on the map.
4. Environmental conditions, such as weather and lighting.
5. Traffic conditions, including the speed and density of surrounding vehicles.
Action Space

The action space encompasses all possible actions the autonomous vehicle can take at any given moment, including:

1. Accelerating or decelerating to change speed.
2. Turning to change direction, which could be quantified into small, medium, or large turns to the left or right.
3. Changing lanes to the left or right.


The transition model describes the probabilities of moving from one state to another, given a specific action. This is complex in autonomous driving due to the inherent uncertainty and dynamic nature of the environment. For example:

1. Taking the action to accelerate might usually result in the car speeding up, but the exact increase in speed could depend on road conditions and whether the vehicle is going uphill or downhill.
2. Turning might not always result in the desired new direction due to slippery roads or unexpected obstacles.

Rewards
The reward function defines the goal of the autonomous vehicle. It assigns a numerical value to each transition, encouraging the vehicle to make decisions that lead to safe, efficient, and law-compliant driving. Possible components of the reward function include:

1. Positive rewards for maintaining a safe distance from other vehicles, obeying traffic laws, and reaching the destination efficiently.
2. Negative rewards (or penalties) for unsafe behavior such as speeding, getting too close to other vehicles, or violating traffic signals.


##Task 3(25 points):

RL is used in various sectors - Healthcare, recommender systems and trading
are a few of those. Pick one of the three areas. Explain one of the problems in any of these
domains that can be more effectively solved by reinforcement learning. Find an open-source
project (if any) that has addressed this problem. Explain this project in detail.

## Answer:

AI Clinician is a notable successful application of RL in health care. Here is the link: https://github.com/matthieukomorowski/AI_Clinician

The AI Clinician uses reinforcement learning to analyze medical data from thousands of past sepsis patients to identify the best treatment strategies. The system aims to optimize the administration of intravenous fluids and vasopressors, which are critical aspects of sepsis management.

The model represents each patient's state using clinical variables (e.g., heart rate, blood pressure, lab test results). The action space includes possible dosages of intravenous fluids and vasopressors. The reward function is designed to maximize the patient's chances of survival and recovery, often based on surrogate markers of clinical improvement or deterioration.

Although specifics can vary, implementation typically involves training an RL model using historical patient data. The model learns a policy mapping patient states to recommended actions (treatment dosages) that maximize the expected reward (patient outcomes).

# Task 4 is for 6000 level ONLY

Task 4(100 points): Implement the game of tic-tac-toe (write a class that implements an agent
playing Tic Tac Toe and learning its Q function) using the Q-learning technique (see the
resources/links provided in class for more details). Clearly describe your evaluation metric and
demonstrate a few runs. You might need to use some online resources to proceed with this. Do
not forget to cite those.

I follow this source: https://github.com/MJeremy2017/reinforcement-learning-implementation/blob/master/TicTacToe/ticTacToe.py

and this post: https://towardsdatascience.com/implement-grid-world-with-q-learning-51151747b455


The Q space for each agent is randomly generated. And they are rewarded 1 if they win, 0 if they tie, and -1 if they lose. For our target agent to train, a draw is not couraged, so 0.1 reward even the game is tie.

My evalution metric: I am using cumultive reward as my reward function. I record the total amount of updates the agent has to their Q matrix at each update, and use that as performance of the agent/algorithm.

The final test is that I run the agent with myself to see how it performs.

In [1]:
import numpy as np
import pickle

BOARD_ROWS = 3
BOARD_COLS = 3


class State:
    def __init__(self, p1, p2):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.boardHash = None
        # init p1 plays first
        self.playerSymbol = 1

    # get unique hash of current board state
    def getHash(self):
        self.boardHash = str(self.board.reshape(BOARD_COLS * BOARD_ROWS))
        return self.boardHash

    def winner(self):
        # row
        for i in range(BOARD_ROWS):
            if sum(self.board[i, :]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[i, :]) == -3:
                self.isEnd = True
                return -1
        # col
        for i in range(BOARD_COLS):
            if sum(self.board[:, i]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[:, i]) == -3:
                self.isEnd = True
                return -1
        # diagonal
        diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)])
        diag_sum2 = sum([self.board[i, BOARD_COLS - i - 1] for i in range(BOARD_COLS)])
        diag_sum = max(abs(diag_sum1), abs(diag_sum2))
        if diag_sum == 3:
            self.isEnd = True
            if diag_sum1 == 3 or diag_sum2 == 3:
                return 1
            else:
                return -1

        # tie
        # no available positions
        if len(self.availablePositions()) == 0:
            self.isEnd = True
            return 0
        # not end
        self.isEnd = False
        return None

    def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    positions.append((i, j))  # need to be tuple
        return positions

    def updateState(self, position):
        self.board[position] = self.playerSymbol
        # switch to another player
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1

    # only when game ends
    def giveReward(self):
        result = self.winner()
        # backpropagate reward
        if result == 1:
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        elif result == -1:
            self.p1.feedReward(0)
            self.p2.feedReward(1)
        else:
            self.p1.feedReward(0.1)
            self.p2.feedReward(0.5)

    # board reset
    def reset(self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 1

    def play(self, rounds=100):
        for i in range(rounds):
            if i % 1000 == 0:
                print("Rounds {}".format(i))
            while not self.isEnd:
                # Player 1
                positions = self.availablePositions()
                p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
                # take action and upate board state
                self.updateState(p1_action)
                board_hash = self.getHash()
                self.p1.addState(board_hash)
                # check board status if it is end

                win = self.winner()
                if win is not None:
                    # self.showBoard()
                    # ended with p1 either win or draw
                    self.giveReward()
                    self.p1.reset()
                    self.p2.reset()
                    self.reset()
                    break

                else:
                    # Player 2
                    positions = self.availablePositions()
                    p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol)
                    self.updateState(p2_action)
                    board_hash = self.getHash()
                    self.p2.addState(board_hash)

                    win = self.winner()
                    if win is not None:
                        # self.showBoard()
                        # ended with p2 either win or draw
                        self.giveReward()
                        self.p1.reset()
                        self.p2.reset()
                        self.reset()
                        break

    # play with human
    def play2(self):
        while not self.isEnd:
            # Player 1
            positions = self.availablePositions()
            p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
            # take action and upate board state
            self.updateState(p1_action)
            self.showBoard()
            # check board status if it is end
            win = self.winner()
            if win is not None:
                if win == 1:
                    print(self.p1.name, "wins!")
                else:
                    print("tie!")
                self.reset()
                break

            else:
                # Player 2
                positions = self.availablePositions()
                p2_action = self.p2.chooseAction(positions)

                self.updateState(p2_action)
                self.showBoard()
                win = self.winner()
                if win is not None:
                    if win == -1:
                        print(self.p2.name, "wins!")
                    else:
                        print("tie!")
                    self.reset()
                    break

    def showBoard(self):
        # p1: x  p2: o
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('-------------')


class Player:
    def __init__(self, name, exp_rate=0.3):
        self.name = name
        self.states = []  # record all positions taken
        self.lr = 0.2
        self.exp_rate = exp_rate
        self.decay_gamma = 0.9
        self.states_value = {}  # state -> value

    def getHash(self, board):
        boardHash = str(board.reshape(BOARD_COLS * BOARD_ROWS))
        return boardHash

    def chooseAction(self, positions, current_board, symbol):
        if np.random.uniform(0, 1) <= self.exp_rate:
            # take random action
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else:
            value_max = -999
            for p in positions:
                next_board = current_board.copy()
                next_board[p] = symbol
                next_boardHash = self.getHash(next_board)
                value = 0 if self.states_value.get(next_boardHash) is None else self.states_value.get(next_boardHash)
                # print("value", value)
                if value >= value_max:
                    value_max = value
                    action = p
        # print("{} takes action {}".format(self.name, action))
        return action

    # append a hash state
    def addState(self, state):
        self.states.append(state)

    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st] = 0
            self.states_value[st] += self.lr * (self.decay_gamma * reward - self.states_value[st])
            reward = self.states_value[st]

    def reset(self):
        self.states = []

    def savePolicy(self):
        fw = open('policy_' + str(self.name), 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()

    def loadPolicy(self, file):
        fr = open(file, 'rb')
        self.states_value = pickle.load(fr)
        fr.close()


class HumanPlayer:
    def __init__(self, name):
        self.name = name

    def chooseAction(self, positions):
        while True:
            row = int(input("Input your action row:"))
            col = int(input("Input your action col:"))
            action = (row, col)
            if action in positions:
                return action

    # append a hash state
    def addState(self, state):
        pass

    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        pass

    def reset(self):
        pass


if __name__ == "__main__":
    # training
    p1 = Player("p1")
    p2 = Player("p2")

    st = State(p1, p2)
    print("training...")
    st.play(50000)

    # play with human
    p1 = Player("computer", exp_rate=0)
    #p1.loadPolicy("policy_p1")

    p2 = HumanPlayer("human")

    st = State(p1, p2)
    st.play2()

training...
Rounds 0
Rounds 1000
Rounds 2000
Rounds 3000
Rounds 4000
Rounds 5000
Rounds 6000
Rounds 7000
Rounds 8000
Rounds 9000
Rounds 10000
Rounds 11000
Rounds 12000
Rounds 13000
Rounds 14000
Rounds 15000
Rounds 16000
Rounds 17000
Rounds 18000
Rounds 19000
Rounds 20000
Rounds 21000
Rounds 22000
Rounds 23000
Rounds 24000
Rounds 25000
Rounds 26000
Rounds 27000
Rounds 28000
Rounds 29000
Rounds 30000
Rounds 31000
Rounds 32000
Rounds 33000
Rounds 34000
Rounds 35000
Rounds 36000
Rounds 37000
Rounds 38000
Rounds 39000
Rounds 40000
Rounds 41000
Rounds 42000
Rounds 43000
Rounds 44000
Rounds 45000
Rounds 46000
Rounds 47000
Rounds 48000
Rounds 49000


FileNotFoundError: [Errno 2] No such file or directory: 'policy_p1'

In [4]:
 # play with human
p1 = Player("computer", exp_rate=0)
#p1.loadPolicy("policy_p1")

p2 = HumanPlayer("human")

st = State(p1, p2)
st.play2()

-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
|   |   | x | 
-------------
Input your action row:1
Input your action col:2
-------------
|   |   |   | 
-------------
|   |   | o | 
-------------
|   |   | x | 
-------------
-------------
|   |   |   | 
-------------
|   |   | o | 
-------------
|   | x | x | 
-------------
Input your action row:3
Input your action col:1
Input your action row:3
Input your action col:0
Input your action row:2
Input your action col:0
-------------
|   |   |   | 
-------------
|   |   | o | 
-------------
| o | x | x | 
-------------
-------------
|   |   |   | 
-------------
|   | x | o | 
-------------
| o | x | x | 
-------------
Input your action row:0
Input your action col:1
-------------
|   | o |   | 
-------------
|   | x | o | 
-------------
| o | x | x | 
-------------
-------------
|   | o |   | 
-------------
| x | x | o | 
-------------
| o | x | x | 
-------------
Input your action row:0
Input your action col:0
-----

# YOu can see it plays pretty good.